<a href="https://colab.research.google.com/github/Poznyakova/-7/blob/main/%D0%94%D0%97_%E2%84%9613%D0%B0_%D0%9F%D0%BE%D0%B7%D0%BD%D1%8F%D0%BA%D0%BE%D0%B2%D0%B0_%D0%9E%D0%BB%D1%8C%D0%B3%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
# загружаем библиотеки
import pandas as pd
import numpy as np

from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, IntegerType, DateType

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession #to define a SparkSession

sc = SparkContext('local') #spark_connection
spark = SparkSession(sc) # open spark session

Загружаем данные

In [ ]:
df = spark.read.format("com.databricks.spark.csv").options(sep=",", header=True, quote="").csv('iris.csv')

In [ ]:
df.show()

Определяем тип данных

In [ ]:
df.printSchema()

Преобразуем датафрейм (преобразуем названия колонок, убираем ковычки, изменяем вид данных)

In [ ]:
newColumns = ["sepal_length","sepal_width","petal_length","petal_width","variety"]
df_new = df.toDF(*newColumns)

df_new = df_new.withColumn("sepal_length", regexp_replace(col("sepal_length"), '"', ""))\
    .withColumn("variety", regexp_replace(col("variety"), '"', ""))

df_new = df_new \
    .withColumn('sepal_length', col('sepal_length').cast(DoubleType())) \
    .withColumn('sepal_width', col('sepal_width').cast(DoubleType())) \
    .withColumn('petal_length', col('petal_length').cast(DoubleType())) \
    .withColumn('petal_width', col('petal_width').cast(DoubleType()))

Выводим статистику по датафрейму

In [ ]:
df_new.summary().show()

Векторизация признаков. Pipeline

In [ ]:
pipeline = Pipeline(stages = 
[
  StringIndexer(inputCol='variety', outputCol='varietyInd'),
  VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol='Features')
]
)

In [ ]:
pipelineTrained = pipeline.fit(df_new)

In [ ]:
pipelineTrained.transform(df_new).show()

In [ ]:
df_features = pipelineTrained.transform(df_new)

Делим датафрейм на обучающую и тестовую выборки (80/20)

In [ ]:
train, test = df_features.randomSplit([0.8, 0.2], seed=12345)

In [ ]:
train.show()

Создание модели и ее обучение

In [ ]:
lr = LogisticRegression(featuresCol = 'Features', labelCol = 'varietyInd')
lrModel = lr.fit(train)

In [ ]:
train_res = lrModel.transform(train)
test_res = lrModel.transform(test)

In [ ]:
train_res.show()

In [ ]:
test_res.show()

MulticlassClassificationEvaluator

In [ ]:
ev = MulticlassClassificationEvaluator(labelCol='varietyInd')

In [ ]:
print('Точность предсказания на обучающей выборке:', ev.evaluate(test_res)*100, '%')

In [ ]:
print('Точность предсказания на обучающей выборке:', ev.evaluate(train_res)*100, '%')